# Interfacing C libraries



## [ctypes](https://docs.python.org/3/library/ctypes.html)

In [1]:
!cat primelib/primelib.h

﻿#ifndef PRIMELIB_H
#define PRIMELIB_H

__declspec(dllexport) int __cdecl is_prime(unsigned long long x);
__declspec(dllexport) unsigned long long __cdecl smallest_divisor(unsigned long long x);

#endif


In [2]:
!cat primelib/primelib.c

﻿#include "primelib.h"

#include "math.h"

__declspec(dllexport) int __cdecl is_prime(unsigned long long x)
{
	if (x == 1) return 0;
	return smallest_divisor(x) == x;
}

__declspec(dllexport) unsigned long long __cdecl smallest_divisor(unsigned long long x)
{
	if (x < 4) return x;
	unsigned long long max_divisor = (unsigned long long)sqrt((float)x) + 1;
	for (unsigned long long i = 2; i < max_divisor; i++) {
		if (x % i == 0) return i;
	}
	return x;
}


### In Windows

In [3]:
from ctypes import windll

primelib = windll.LoadLibrary(".\\primelib.dll")
primelib.is_prime


In [4]:
primelib.is_prime(2)

1

Cool!

In [7]:
primelib.is_prime(22801763489)

ArgumentError: argument 1: <class 'OverflowError'>: int too long to convert

Not so cool. But it should work, ehm...

In [5]:
from ctypes import c_ulonglong

primelib.is_prime.argtypes = [c_ulonglong]
primelib.smallest_divisor.argtypes = [c_ulonglong]
primelib.smallest_divisor.restype = c_ulonglong

{x : primelib.smallest_divisor(x) for x in range(10**12, 10**12+20)}

{1000000000000: 2,
 1000000000001: 73,
 1000000000002: 2,
 1000000000003: 61,
 1000000000004: 2,
 1000000000005: 3,
 1000000000006: 2,
 1000000000007: 34519,
 1000000000008: 2,
 1000000000009: 29,
 1000000000010: 2,
 1000000000011: 3,
 1000000000012: 2,
 1000000000013: 7,
 1000000000014: 2,
 1000000000015: 5,
 1000000000016: 2,
 1000000000017: 3,
 1000000000018: 2,
 1000000000019: 1601}

### In Linux

In [3]:
from ctypes import cdll

primelib = cdll.LoadLibrary("./libprimelib.so")

In [4]:
from ctypes import c_ulonglong

primelib.is_prime.argtypes = [c_ulonglong]
primelib.smallest_divisor.argtypes = [c_ulonglong]
primelib.smallest_divisor.restype = c_ulonglong

In [6]:
{x : primelib.smallest_divisor(x) for x in range(10**12, 10**12+20)}

{1000000000000: 2,
 1000000000001: 73,
 1000000000002: 2,
 1000000000003: 61,
 1000000000004: 2,
 1000000000005: 3,
 1000000000006: 2,
 1000000000007: 34519,
 1000000000008: 2,
 1000000000009: 29,
 1000000000010: 2,
 1000000000011: 3,
 1000000000012: 2,
 1000000000013: 7,
 1000000000014: 2,
 1000000000015: 5,
 1000000000016: 2,
 1000000000017: 3,
 1000000000018: 2,
 1000000000019: 1601}

## [cffi](https://cffi.readthedocs.io/en/latest/)

In [16]:
import cffi

ffi = cffi.FFI()

ffi.cdef("""
    int is_prime(unsigned long long);
    unsigned long long smallest_divisor(unsigned long long);
""")

ffi.set_source("_primelib_cffi",
"""
     #include "primelib/primelib.h"
""",
    libraries=['primelib'],
    library_dirs =["."]
)

ffi.compile(verbose=True)

generating ./_primelib_cffi.c
(already up-to-date)
the current directory is '/home/honza/hilase-python-course-2021'
running build_ext
building '_primelib_cffi' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/home/honza/hilase-python-course-2021/.venv/include -I/usr/include/python3.8 -c _primelib_cffi.c -o ./_primelib_cffi.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 ./_primelib_cffi.o -L. -lprimelib -o ./_primelib_cffi.cpython-38-x86_64-linux-gnu.so


'/home/honza/hilase-python-course-2021/_primelib_cffi.cpython-38-x86_64-linux-gnu.so'

In [26]:
from _primelib_cffi.lib import is_prime, smallest_divisor

In [31]:
smallest_divisor(44864561566508441)

7161929